## Using Python to interact with the OGC API Features service

### 1. Imports and connectivity check

##### 1.1 Importing Library Dependencies

In [ ]:
# Import the Folium library, which is used for creating interactive maps in Python.
import folium

# Import the GeoPandas library, which is used for working with geospatial data in Python.
import geopandas as gpd

# Import the JSON library, which allows you to work with JSON data in Python.
import json

# Import the Matplotlib library, which is commonly used for creating static, interactive, and animated plots and charts.
import matplotlib.pyplot as plt

# Import the NumPy library, which provides support for large, multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these arrays.
import numpy as np

# Import the Features class from the owslib.ogcapi.features module, which is used for working with OGC API Features, a web-based geospatial data service.
from owslib.ogcapi.features import Features

# Import the requests library, which is used for making HTTP requests in Python, enabling you to retrieve data from web services or APIs.
import requests


##### 1.2 To check PyGeoAPI connectivity from the notebook environment

In [ ]:
# To check whether the pygeoapi server is running
# Set the endpoint URL following the format: http://pygeoapi:<port_number>

# If you map the internal port 80 of the pygeoapi server to port 5000 of your host machine, 
# the server won't be reachable via the localhost address from another docker container. 
# The reason for this is that every single Docker container has it's own localhost. 
# So if your Jupyter Notebook container aims to request a service with a localhost address, it will request the
# service on it's internal host and not the host of your computer/server.
# So the strategy we were using for browser access or creating a connection from QGIS software, will not work here.
# To be precise http://localhost:5000 url will not work in this case.
# 
# Fortunately, Docker has a dedicated networking concept, which supports networking between containers. 
# There is a great tutorial that describes how to network your containers: 
# https://docs.docker.com/engine/tutorials/networkingcontainers/

endpoint = "http://pygeoapi:80"
response = requests.get(endpoint)
if response.status_code == 200:
    print("pygeoapi server is running!")
else:
    print("pygeoapi server is not running.")


### 2. Data access 

##### 2.1 To list the collections available in the server instance

In [ ]:
# Initialize a connection to a GeoSpatial API at 'http://pygeoapi:80',
# retrieve a list of available data collections (e.g., datasets, layers),
# and print the collections in JSON format with proper indentation.
feature_dataset = Features('http://pygeoapi:80')
collections = feature_dataset.collections() 
print(json.dumps(collections, indent=4))

##### 2.2 To access the landing page of a specific collection; that provides you with the metadata of the collection

In [ ]:

protected_sites = feature_dataset.collection('NSG')
print(json.dumps(protected_sites, indent=2))

##### 2.3 View the queryable parameters and their datatypes

In [ ]:
# View the queryable parameters available in the dataset.
protected_sites_queryables = feature_dataset.collection_queryables('NSG')
print(json.dumps(protected_sites_queryables, indent=2))

##### 2.4 Access items (Features) of the NSG Feature Collection

In [ ]:
protected_sites_items = feature_dataset.collection_items('NSG') 
print(json.dumps(protected_sites_items, indent=4))

# Note: The default paging strategy is set to 10; so only the first 10 Features are returned

##### 2.5 Assign the response of a feature request to a variable

In [ ]:
# Store the queried feature items
protected_sites_features = protected_sites_items['features']

# To access a feature from the stored variable
protected_sites_features[0]

##### 2.6 To access items of a dataset using a customized PyGeoAPI endpoint

In [ ]:
# In the following access url I am setting the page limit from 10 to 100. Now 100 features in NSG dataset can be obsererved 
# in the response. Using the same url signature one can explore the queryables to filter out a subset of information. 
# For example; after ? if we add VOLLZUG=Landkreis Cloppenburg; the resulting response wiil select the 
# NSGs that are under the jurisdiction of the Cloppenburg district.

pygeoapi_url_to_access_dataset = "http://pygeoapi:80/collections/NSG/items?limit=1000"

# Make an HTTP GET request to fetch the GeoJSON data
response = requests.get(pygeoapi_url_to_access_dataset)
nsg_json_data = response.json()
print(nsg_json_data)
# This may take 10-15 seconds

##### 2.7 Loading the data in a Geopandas Dataframe

In [ ]:
# Create a GeoDataFrame directly from the GeoJSON data
# Advantages of using a GeoPandas DataFrame:
# 1. Geospatial Operations: Easily perform geospatial operations such as spatial joins, buffers, and overlays.
# 2. Data Visualization: Seamlessly visualize geographic data with built-in plotting capabilities.
# 3. Interoperability: GeoPandas supports multiple geospatial formats, making it easy to work with various data sources.
# 4. Integration: GeoPandas integrates geospatial data with tabular data, simplifying analysis.
# 5. Data Cleaning: Another important capability we get with Geopandas is an opportunity to remove redundant columns and records thats are outdated,
#    or contains error or is not relevant to the present data analysis task under consideration. Python provides this data cleaning 
#    capability with its Pandas library which has been extended to Geopandas as well.

nsg_data = gpd.GeoDataFrame.from_features(nsg_json_data['features'])
print(nsg_data)

### 3. Data Visualizations with Interactive Plots

##### 3.1 Plotting the Geomemtries for the NSG Dataset on the leaflet basemap

In [ ]:
# Set the CRS for your GeoDataFrame (assuming it's EPSG 4647)
nsg_data.crs = 'EPSG:4647'

# Create a base map centered around the geometries
map_center = nsg_data.union_all().centroid
m = folium.Map(location=[map_center.y, map_center.x], zoom_start=9)

# Style function to make the geometries deep blue
def style_function(feature):
    return {
        'fillColor': 'red',  # Fill color
        'color': 'red',      # Border color
        'weight': 2,                # Border width
        'fillOpacity': 0.6          # Fill opacity
    }

# Iterate through the GeoDataFrame and add geometries to the map
for index, row in nsg_data.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=style_function
    ).add_to(m)

# Save the map to an HTML file or display it in a Jupyter Notebook
# m.save('interactive_map.html')  # Save as an HTML file
m 
# This may take 10-15 seconds

##### 3.2 Using a spatial filter to query out datapoints within the Municipality of Lorup

In [ ]:

# Specify the URL of your PyGeoAPI feature dataset with a spatial filter 'bbox'
pygeoapi_url_to_access_queried_dataset_for_lorup = "http://pygeoapi:80/collections/NSG/items?bbox=7.4460,52.8722,7.8532,53.0138"
# Make an HTTP GET request to fetch the GeoJSON data
response = requests.get(pygeoapi_url_to_access_queried_dataset_for_lorup)
nsg_json_data_lorup = response.json()
nsg_feature_data_lorup = gpd.GeoDataFrame.from_features(nsg_json_data_lorup['features'])
print(nsg_feature_data_lorup)


In [ ]:
# Set the CRS for your GeoDataFrame (assuming it's EPSG 4647)
nsg_feature_data_lorup.crs = 'EPSG:4647'

# Create a base map centered around the geometries
map_center = nsg_feature_data_lorup.union_all().centroid
m = folium.Map(location=[map_center.y, map_center.x], zoom_start=9)

# Style function to make the geometries deep blue
def style_function(feature):
    return {
        'fillColor': 'red',  # Fill color
        'color': 'red',      # Border color
        'weight': 2,                # Border width
        'fillOpacity': 0.6          # Fill opacity
    }

# Iterate through the GeoDataFrame and add geometries to the map
for index, row in nsg_feature_data_lorup.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=style_function
    ).add_to(m)

# Save the map to an HTML file or display it in a Jupyter Notebook
# m.save('interactive_map.html')  # Save as an HTML file
m 
# This may take 10-15 seconds

##### 3.3 Adding buffer zone around the geometries for better data visualization

In [ ]:
nsg_feature_data_lorup.crs = 'EPSG:4647'

# Create a base map centered around the geometries
map_center = nsg_feature_data_lorup.union_all().centroid
m = folium.Map(location=[map_center.y, map_center.x], zoom_start=9)

# Style function for the original geometries
def style_function_geom(feature):
    return {
        'fillColor': 'red',  # Fill color for geometries
        'color': 'red',      # Border color for geometries
        'weight': 2,                # Border width for geometries
        'fillOpacity': 0.6          # Fill opacity for geometries
    }

# Style function for the buffer areas
def style_function_buffer(feature):
    return {
        'fillColor': 'red',         # Fill color for buffer areas
        'color': 'red',             # Border color for buffer areas
        'weight': 2,                # Border width for buffer areas
        'fillOpacity': 0.6          # Fill opacity for buffer areas
    }

# Buffer function to create a buffer around the geometries
def create_buffer(geometry, buffer_distance):
    return geometry.buffer(buffer_distance)

# Iterate through the GeoDataFrame, add buffer areas, and add them to the map
buffer_distance = 0.02  # Adjust the buffer distance as needed
for index, row in nsg_feature_data_lorup.iterrows():
    buffered_geometry = create_buffer(row['geometry'], buffer_distance)
    
    # Add the original geometries with style
    folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': row['geometry'].__geo_interface__,
            'properties': {},
        },
        style_function=style_function_geom
    ).add_to(m)

    # Add the buffered geometries with style
    folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': buffered_geometry.__geo_interface__,
            'properties': {},
        },
        style_function=style_function_buffer
    ).add_to(m)

# Save the map to an HTML file or display it in a Jupyter Notebook
# m.save('interactive_map_with_buffer.html')  # Save as an HTML file
m

# Note: The buffer zones has multiple practical usecases. For example, let us assume that we have been assigned an objective to figure
# out regions suitable for establishing wind farms. Now, one important constraint for setting up such a farm is to keep it away from
# wild life, reserve forests, sanctuaries and protected sites. The constraint is imposed to ensure that the wind farms do not affect
# the wild life and vegetation in the particular region. In such a scenario it becomes inevitable to maintain a certain buffer region 
# around the protected sites; so that while selecting and shortlisting areas suitable for wind farm installation  the concerned person 
# can discard the proposed regions which intersects with the buffer zone geometries.    

**End of this exercise, please continue with the PDF/H5P content of the lerning material..**